In [1]:
import streamlit as st
import time
import re
import random
from pymongo.mongo_client import MongoClient
import pandas as pd
import json
import random
import tqdm as tqdm
import string
from datetime import datetime


In [2]:
uri = st.secrets["mongo"]['uri']
client = MongoClient(uri)
db = client["Replication"]
collexp_code = db.exp_code

In [3]:
def getRecord(key):
    # Query for documents where the key exists
    documents = collexp_code.find({key: {"$exists": True}})
    # Iterate over the documents and print the values for the key
    for doc in documents:
        if key in doc:
            return doc[key]
        else:
            return None

In [4]:
# Retrieve a sample of documents
documents = collexp_code.find()  # Adjust the limit as needed

# Extract keys from the documents
keys = []
for document in documents:
    keys.append(document.keys())

keys = [list(key)[1] for key in keys]

#store each record
rec = []
for key in tqdm.tqdm(keys):
    rec.append(json.loads(getRecord(key)))


100%|██████████| 300/300 [00:58<00:00,  5.10it/s]


In [5]:
#normalise all records to dataframe
df = pd.json_normalize(json.loads(json.dumps(rec)))

In [6]:
#data processing processing dialogue and extracting choice data, response time,
# overall interaction time, and choice correspondance
def analyse_chat(chatObject, expcode, chat):
    
    SCN = expcode.split('_')[1].split('&')[chat].split("$")[0]
    CND = expcode.split('_')[1].split('&')[chat].split("$")[1]

    time_format = "%Y-%m-%d %H:%M:%S"
    choiceCorr = chatObject[-2]['content']
    overall_time_a = chatObject[0]['timestamp']
    overall_time_b = chatObject[-1]['timestamp']
    response_time_a = chatObject[-3]['timestamp']
    response_time_b = chatObject[-2]['timestamp']

    overall_time_a = datetime.strptime(overall_time_a, time_format)
    overall_time_b = datetime.strptime(overall_time_b, time_format)
    overall_time = (overall_time_b - overall_time_a).total_seconds()

    response_time_a = datetime.strptime(response_time_a, time_format)
    response_time_b = datetime.strptime(response_time_b, time_format)
    choiceCorr_time = (response_time_b - response_time_a).total_seconds()
   

    return [SCN, CND, choiceCorr, choiceCorr_time, overall_time]

def formated_data(chat1, chat2, chat3, expcode):

    a = analyse_chat(chat1, expcode=expcode, chat=0)
    b = analyse_chat(chat2, expcode=expcode, chat=1)
    c = analyse_chat(chat3, expcode=expcode, chat=2)

    sc1_cond= []
    sc2_cond = []
    sc3_cond = []

    for ar in [a, b, c]: # I am not quite confident about this chunck of code.
        if ar[0] == 'BA':
            sc1_cond= ar[1:]
        if ar[0] == 'IP':
            sc2_cond= ar[1:]
        if ar[0] == 'CJ':
            sc3_cond= ar[1:]

    return sc1_cond + sc2_cond + sc3_cond

In [7]:
ndf = pd.DataFrame([formated_data(df['chat1'][i], df['chat2'][i], df['chat3'][i], df['expcode'][i])
            for i in range(0, len(df))], columns=["sc1_cond", "sc1_choice", "sc1_ctime", "sc1_otime",
                                                  "sc2_cond", "sc2_choice", "sc2_ctime", "sc2_otime",
                                                  "sc3_cond", "sc3_choice", "sc3_ctime", "sc_otime"
])

In [8]:
# Find the index of the column after which to insert the new column
insert_loc = df.columns.get_loc('chat3') + 1
# Split df1 into two parts
df1_part1 = df.iloc[:, :insert_loc]
df1_part2 = df.iloc[:, insert_loc:]

df_demographics = pd.read_csv("demographics.csv")

# Concatenate the parts and df2
result = pd.concat([df1_part1, ndf, df1_part2, df_demographics.iloc[:, 1:]], axis=1)

In [9]:
result.to_csv('final_data.csv', sep=',', encoding='utf-8')